<a href="https://colab.research.google.com/github/DaheePark0415/Econ512-Fall2024/blob/main/IO_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1(a).

In [29]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Load the data
df = pd.read_csv('product_data.csv')

# Check the structure of the data to ensure it has the correct columns
print(df.head())

   product_ID  nest     price     sugar  caffeine  market_share  \
0           1  Diet  2.814362  0.631224  6.752525      0.111141   
1           2  Diet  2.935735  0.004553  6.784396      0.081787   
2           3  Diet  2.467309  0.739947  5.761261      0.085727   
3           4  Diet  1.543958  0.103660  4.468299      0.007187   
4           5  Diet  1.495961  0.971926  4.052750      0.015912   

   caffeine_extract_price  corn_syrup_price  t  
0                0.267468          0.251714  1  
1                0.320000          0.253146  1  
2                0.252531          0.314781  1  
3                0.203220          0.227481  1  
4                0.156466          0.244453  1  


In [30]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from scipy.optimize import minimize


In [31]:
# Load the dataset
df = pd.read_csv('/content/product_data.csv')

# Show the first few rows of the dataset to verify
print(df.head())


   product_ID  nest     price     sugar  caffeine  market_share  \
0           1  Diet  2.814362  0.631224  6.752525      0.111141   
1           2  Diet  2.935735  0.004553  6.784396      0.081787   
2           3  Diet  2.467309  0.739947  5.761261      0.085727   
3           4  Diet  1.543958  0.103660  4.468299      0.007187   
4           5  Diet  1.495961  0.971926  4.052750      0.015912   

   caffeine_extract_price  corn_syrup_price  t  
0                0.267468          0.251714  1  
1                0.320000          0.253146  1  
2                0.252531          0.314781  1  
3                0.203220          0.227481  1  
4                0.156466          0.244453  1  


In [35]:
# Define the utility function with corrected handling for Diet and Regular using the 'nest' column
def utility(params, price, sugar, caffeine, nest):
    alpha, beta1, beta2, gamma_d, gamma_r = params
    diet = (nest == 'Diet').astype(int)
    regular = (nest != 'Diet').astype(int)  # Assuming the other category is Regular
    return alpha * price + beta1 * sugar + beta2 * caffeine + gamma_d * diet + gamma_r * regular

# Define the log-likelihood function for the multinomial logit model
def log_likelihood(params, df):
    # Calculate utility for each product in each time period
    df['utility'] = utility(params, df['price'], df['sugar'], df['caffeine'], df['nest'])

    # Exponentiate utility
    df['exp_utility'] = np.exp(df['utility'])

    # Group by time period, sum over products to get the denominator for each period
    df['sum_exp_util'] = df.groupby('t')['exp_utility'].transform('sum') + 1  # Add 1 for the outside option

    # Probability of choosing product j at time t
    df['prob'] = df['exp_utility'] / df['sum_exp_util']

    # Log of the probability weighted by quantity (here using 'market_share')
    df['log_prob'] = np.log(df['prob']) * df['market_share']

    # Negative log-likelihood (since we minimize)
    return -df['log_prob'].sum()

# Initialize the parameters: [alpha, beta1, beta2, gamma_d, gamma_r]
initial_guess = [0.1, 0.1, 0.1, 0.1, 0.1]

# Estimate the parameters by minimizing the negative log-likelihood
result = minimize(log_likelihood, initial_guess, args=(df,), method='BFGS')

# Print the estimated parameters
print("Estimated parameters:", result.x)


Estimated parameters: [-0.32866229  0.85382714  0.8080273  11.61252541  8.8465584 ]


# 1(b).

In [36]:
from sklearn.linear_model import LinearRegression

# Define the exogenous variables (instruments + other exogenous variables)
exog_vars = ['sugar', 'caffeine', 'caffeine_extract_price', 'corn_syrup_price']

# First stage: Regress price on instruments and other exogenous variables
model_stage_1 = LinearRegression()
model_stage_1.fit(df[exog_vars], df['price'])

# Get the predicted price from the first stage
df['predicted_price'] = model_stage_1.predict(df[exog_vars])


In [37]:
# Adjust the utility function to use the predicted price
def utility_iv(params, predicted_price, sugar, caffeine, nest):
    alpha, beta1, beta2, gamma_d, gamma_r = params
    diet = (nest == 'Diet').astype(int)
    regular = (nest != 'Diet').astype(int)
    return alpha * predicted_price + beta1 * sugar + beta2 * caffeine + gamma_d * diet + gamma_r * regular

# Adjust the log-likelihood function to use the predicted price
def log_likelihood_iv(params, df):
    # Calculate utility for each product in each time period using predicted price
    df['utility'] = utility_iv(params, df['predicted_price'], df['sugar'], df['caffeine'], df['nest'])

    # Exponentiate utility
    df['exp_utility'] = np.exp(df['utility'])

    # Group by time period, sum over products to get the denominator for each period
    df['sum_exp_util'] = df.groupby('t')['exp_utility'].transform('sum') + 1  # Add 1 for the outside option

    # Probability of choosing product j at time t
    df['prob'] = df['exp_utility'] / df['sum_exp_util']

    # Log of the probability weighted by market share
    df['log_prob'] = np.log(df['prob']) * df['market_share']

    # Negative log-likelihood (since we minimize)
    return -df['log_prob'].sum()

# Initialize the parameters: [alpha, beta1, beta2, gamma_d, gamma_r]
initial_guess_iv = [0.1, 0.1, 0.1, 0.1, 0.1]

# Estimate the parameters by minimizing the negative log-likelihood
result_iv = minimize(log_likelihood_iv, initial_guess_iv, args=(df,), method='BFGS')

# Print the estimated parameters
print("Estimated parameters (IV):", result_iv.x)


Estimated parameters (IV): [-1.33114723  1.03226555  1.02240006 14.35370063 12.09851264]


Problem 1(b) involves addressing endogeneity in the price variable. In the given context, the price of soft drinks is correlated with unobserved quality (𝜉𝑗𝑡​). This means that standard OLS estimators would be biased. To solve this issue, we can use instrumental variables (IV) for the price.

In this problem, the suggested instruments are:

Caffeine Extract Price and Corn Syrup Price.

These instruments are used because they affect the price but are not directly related to the unobserved demand shock (𝜉𝑗𝑡).

Approach:
Two-Stage Least Squares (2SLS): First, we will regress the endogenous variable (price) on the instruments to get the predicted prices. Then, we will use these predicted prices in the utility function to estimate the parameters.

Conditions for Valid Instruments:

Relevance: The instruments must be correlated with the endogenous variable (price).
Exogeneity: The instruments must not be correlated with the error term (unobserved quality 𝜉𝑗𝑡).

# 1(c)

roblem 1(c) involves calculating the own-price derivatives and own-price elasticities for the products. These derivatives and elasticities are important for understanding how the market share of each product responds to changes in its own price.

In [38]:
# Get the estimated alpha from the previous results
alpha_est = result_iv.x[0]  # Assuming alpha is the first estimated parameter from the IV model

# Function to calculate own-price derivatives
def own_price_derivative(alpha, market_share):
    return alpha * market_share * (1 - market_share)

# Function to calculate own-price elasticity
def own_price_elasticity(alpha, price, market_share):
    return own_price_derivative(alpha, market_share) * (price / market_share)

# Calculate own-price derivatives and elasticities for each product in each time period
df['own_price_derivative'] = own_price_derivative(alpha_est, df['market_share'])
df['own_price_elasticity'] = own_price_elasticity(alpha_est, df['price'], df['market_share'])

# Print the mean own-price elasticity for Regular and Diet drinks
mean_elasticity_diet = df[df['nest'] == 'Diet']['own_price_elasticity'].mean()
mean_elasticity_regular = df[df['nest'] != 'Diet']['own_price_elasticity'].mean()

print(f"Mean Own-Price Elasticity for Diet Drinks: {mean_elasticity_diet}")
print(f"Mean Own-Price Elasticity for Regular Drinks: {mean_elasticity_regular}")


Mean Own-Price Elasticity for Diet Drinks: -2.807415569584785
Mean Own-Price Elasticity for Regular Drinks: -3.9648574224575888


# 1(d).

In [39]:
# Function to calculate cross-price derivatives
def cross_price_derivative(alpha, market_share_j, market_share_k):
    return -alpha * market_share_j * market_share_k

# Function to calculate cross-price elasticity
def cross_price_elasticity(alpha, price_k, market_share_j, market_share_k):
    return cross_price_derivative(alpha, market_share_j, market_share_k) * (price_k / market_share_j)

# Calculate cross-price elasticities between product 1 and all other products
def calculate_cross_price_elasticities(df, product_id):
    # Get the market share and price for the specific product (product 1 in this case)
    product_1_price = df.loc[df['product_ID'] == product_id, 'price'].values[0]
    product_1_share = df.loc[df['product_ID'] == product_id, 'market_share'].values[0]

    # Calculate cross-price elasticities for all other products
    df['cross_price_elasticity'] = df.apply(
        lambda row: cross_price_elasticity(alpha_est, product_1_price, row['market_share'], product_1_share)
        if row['product_ID'] != product_id else np.nan, axis=1)

    return df[['product_ID', 'cross_price_elasticity']]

# Calculate cross-price elasticities for product 1 with all other products
cross_price_elasticities_df = calculate_cross_price_elasticities(df, product_id=1)

# Print the cross-price elasticities for product 1
print(cross_price_elasticities_df)

# Calculate the mean cross-price elasticity for diet and regular drinks
mean_cross_elasticity_diet = cross_price_elasticities_df[df['nest'] == 'Diet']['cross_price_elasticity'].mean()
mean_cross_elasticity_regular = cross_price_elasticities_df[df['nest'] != 'Diet']['cross_price_elasticity'].mean()

print(f"Mean Cross-Price Elasticity between Product 1 and Diet Drinks: {mean_cross_elasticity_diet}")
print(f"Mean Cross-Price Elasticity between Product 1 and Regular Drinks: {mean_cross_elasticity_regular}")


     product_ID  cross_price_elasticity
0             1                     NaN
1             2                0.416372
2             3                0.416372
3             4                0.416372
4             5                0.416372
..          ...                     ...
995           6                0.416372
996           7                0.416372
997           8                0.416372
998           9                0.416372
999          10                0.416372

[1000 rows x 2 columns]
Mean Cross-Price Elasticity between Product 1 and Diet Drinks: 0.4163720940885115
Mean Cross-Price Elasticity between Product 1 and Regular Drinks: 0.4163720940885114


# 1(e).

roblem 1(e) involves writing a function to generate the Jacobian matrix of price derivatives for a given time period. This matrix, Δ(p), contains the derivatives of the market shares of all products with respect to the prices of all products in the same time period.

Jacobian Matrix of Price Derivatives
The Jacobian matrix Δ(p) is a square matrix where each element Δ𝑖𝑗 is the derivative of the market share of product 𝑗j with respect to the price of product 𝑖:

The diagonal elements
Δ𝑗𝑗  are the own-price derivatives:
Δ𝑗𝑗=∂𝑠𝑗/∂𝑝𝑗=α⋅sj​ ⋅(1−sj)j

The off-diagonal elements
Δ𝑖𝑗   are the cross-price derivatives:
Δ𝑖𝑗=∂𝑠𝑗∂𝑝𝑖=−α⋅sj​ ⋅si​ for i!=j


In [40]:
import numpy as np

# Function to compute the Jacobian matrix of price derivatives for a given period
def jacobian_matrix(time_period, df, alpha):
    # Filter the dataframe for the given time period
    df_period = df[df['t'] == time_period]

    # Get the number of products in this period
    num_products = df_period.shape[0]

    # Initialize the Jacobian matrix (num_products x num_products)
    jacobian = np.zeros((num_products, num_products))

    # Loop over all pairs of products to compute the Jacobian elements
    for i in range(num_products):
        for j in range(num_products):
            # Extract market shares for product i and product j
            s_i = df_period.iloc[i]['market_share']
            s_j = df_period.iloc[j]['market_share']

            if i == j:
                # Own-price derivative (diagonal element)
                jacobian[i, j] = alpha * s_j * (1 - s_j)
            else:
                # Cross-price derivative (off-diagonal element)
                jacobian[i, j] = -alpha * s_j * s_i

    return jacobian

# Calculate the Jacobian matrix for the last time period (t = 100)
time_period = 100  # Last period
alpha_est = result_iv.x[0]  # Alpha parameter from the IV model (estimated)

# Generate the Jacobian matrix for the given period
jacobian_last_period = jacobian_matrix(time_period, df, alpha_est)

# Print the Jacobian matrix for the last time period
print(f"Jacobian matrix for time period {time_period}:")
print(jacobian_last_period)


Jacobian matrix for time period 100:
[[-7.84978749e-02  1.16387376e-04  8.55287265e-04  1.70211968e-03
   2.05111003e-02  3.27488772e-03  3.72503045e-02  8.06115650e-03
   3.41472680e-03  3.02690381e-03]
 [ 1.16387376e-04 -1.84689472e-03  1.88830417e-05  3.75794172e-05
   4.52844299e-04  7.23030073e-05  8.22412636e-04  1.77974302e-04
   7.53903762e-05  6.68280157e-05]
 [ 8.55287265e-04  1.88830417e-05 -1.34522567e-02  2.76157071e-04
   3.32778326e-03  5.31327738e-04  6.04360264e-03  1.30786654e-03
   5.54015657e-04  4.91094075e-04]
 [ 1.70211968e-03  3.75794172e-05  2.76157071e-04 -2.64981054e-02
   6.62267009e-03  1.05740309e-03  1.20274619e-02  2.60280430e-03
   1.10255465e-03  9.77333493e-04]
 [ 2.05111003e-02  4.52844299e-04  3.32778326e-03  6.62267009e-03
  -2.46128183e-01  1.27420540e-02  1.44934860e-01  3.13646453e-02
   1.32861451e-02  1.17771891e-02]
 [ 3.27488772e-03  7.23030073e-05  5.31327738e-04  1.05740309e-03
   1.27420540e-02 -5.00054557e-02  2.31409035e-02  5.00780993e